# Scikit-Learn IRIS Model using jsonData

**Goal**: Wrap a scikit-learn python model for use as a prediction microservice in seldon-core

#### Steps

1. Run locally on Docker to test
2. Deploy on seldon-core running on a kubernetes cluster

### Setup Python

In [33]:
%%writefile requirements.txt
scikit-learn
spacy
dill
pandas
sklearn
seldon-core
pycurl

Overwriting requirements.txt


In [10]:
!python -m venv venv

In [34]:
!source ./venv/bin/activate

In [12]:
!mkdir /home/jovyan/.config/pip

mkdir: cannot create directory ‘/home/jovyan/.config/pip’: File exists


In [13]:
%%writefile /home/jovyan/.config/pip/pip.conf
[global]
index-url = https://jfrog.aaw.cloud.statcan.ca/artifactory/api/pypi/pypi-remote/simple

Overwriting /home/jovyan/.config/pip/pip.conf


In [35]:
%%capture
!pip install -r requirements.txt

In [15]:
%%capture
!python -m spacy download en_core_web_sm

In [16]:
!python -m ipykernel install --user --name=sklearn_iris_jsondata

Installed kernelspec sklearn_iris_jsondata in /etc/share/jupyter/kernels/sklearn_iris_jsondata


## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html) to setup Seldon Core with an ingress - either Ambassador or Istio.

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:

 * Ambassador: `kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080`
 * Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:80`

In [1]:
# we use istio
#!kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080

error: error executing jsonpath "{.items[0].metadata.name}": Error executing template: array index out of bounds: index 0, length 0. Printing more information for debugging the template:
	template was:
		{.items[0].metadata.name}
	object given to jsonpath engine was:
		map[string]interface {}{"apiVersion":"v1", "items":[]interface {}{}, "kind":"List", "metadata":map[string]interface {}{"resourceVersion":"", "selfLink":""}}


error: TYPE/NAME and list of ports are required for port-forward
See 'kubectl port-forward -h' for help and examples


In [42]:
!kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:80

Error from server (Forbidden): pods is forbidden: User "system:serviceaccount:bryanpaget:default-editor" cannot list resource "pods" in API group "" in the namespace "istio-system"
error executing jsonpath "{.items[0].metadata.name}": Error executing template: array index out of bounds: index 0, length 0. Printing more information for debugging the template:
	template was:
		{.items[0].metadata.name}
	object given to jsonpath engine was:
		map[string]interface {}{"apiVersion":"v1", "items":[]interface {}{}, "kind":"List", "metadata":map[string]interface {}{"resourceVersion":"", "selfLink":""}}



error: TYPE/NAME and list of ports are required for port-forward
See 'kubectl port-forward -h' for help and examples


In [31]:
#!kubectl create namespace seldon
# I'll use my namespace, bryanpaget

In [30]:
# not necessary on dev
#!kubectl config set-context $(kubectl config current-context) --namespace=bryanpaget

error: current-context is not set
error: you must specify a non-empty context name or --current


Create a seldon config file to deploy the containerized image you just created

In [37]:
%%writefile sklearn_iris_jsondata_deployment.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: seldon-deployment-example-3
  namespace: bryanpaget
spec:
  name: sklearn-iris-deployment
  annotations:
    sidecar.istio.io/inject: "false"
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/sklearn-iris:0.3
          imagePullPolicy: IfNotPresent
          name: sklearn-iris-classifier
    graph:
      children: []
      endpoint:
        type: REST
      name: sklearn-iris-classifier
      type: MODEL
    name: sklearn-iris-predictor
    replicas: 1

Overwriting sklearn_iris_jsondata_deployment.yaml


In [38]:
!kubectl create -f sklearn_iris_jsondata_deployment.yaml

seldondeployment.machinelearning.seldon.io/seldon-deployment-example-3 created


In [39]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=seldon-deployment-example -o jsonpath='{.items[0].metadata.name}')

deployment "seldon-92a927e5e90d7602e08ba9b9304f70e8" successfully rolled out


### Test by sending prediction calls

jsonData sent as json

In [41]:
import json
import pycurl
import requests
from urllib.parse import urlencode


c = pycurl.Curl()
c.setopt(c.URL, "http://localhost:8003/seldon/seldon/seldon-deployment-example/api/v0.1/predictions")

post_data = {"jsonData": {
    "measurements": {
        "names": ["sepal_length",
                  "sepal_width",
                  "petal_length",
                  "petal_width"],
        "values": [[
            7.233,
            4.652,
            7.39,
            0.324]]
    }
}}

# encoding the string to be used as a query
postfields = urlencode(post_data)

# setting the cURL for POST operation
c.setopt(c.POSTFIELDS, postfields)

# perform file transfer
c.perform()

# Ending the session and freeing the resources
c.close()

print('Response Code: %d' % c.getinfo(c.RESPONSE_CODE))

error: (7, 'Failed to connect to localhost port 8003 after 0 ms: Connection refused')

In [7]:

res=!curl -s -H "Content-Type: application/json" \
    -d  \

j=json.loads(res[0])

print(j)

assert(j["data"]["tensor"]["values"][1]>0.0)

TypeError: the JSON object must be str, bytes or bytearray, not SList

jsonData sent as form-data

In [12]:
res=!curl -s -H 'Content-Type:multipart/form-data' \
    -F jsonData='{"some_data": {"names": ["sepal_length","sepal_width","petal_length","petal_width"],"some_ndarray": [[7.233,4.652,7.39,0.324]]}}' \
    "http://localhost:8003/seldon/seldon/seldon-deployment-example/api/v0.1/predictions"
j=json.loads(res[0])
print(j)
assert(j["data"]["tensor"]["values"][1]>0.0)

{'data': {'names': ['t:0', 't:1', 't:2'], 'tensor': {'shape': [1, 3], 'values': [0.0011707012599976396, 0.7876686831680301, 0.21116061557197244]}}, 'meta': {}}


In [13]:
!kubectl delete -f sklearn_iris_jsondata_deployment.yaml

seldondeployment.machinelearning.seldon.io "seldon-deployment-example" deleted
